Load News Data

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('../data/raw_analyst_ratings.csv')
df.head()

#droping nameless index column
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])